# REGRESIÓN LOGÍSTICA (RL) - BASELINE 

Objetivo: Establecer un modelo baseline usando Regresión Logística para comparar con los modelos de Gradient Boosting.

Fases temporales:
- T0 (Matrícula)        : Variables disponibles al momento de inscripción
- T1 (Fin 1er Semestre) : T0 + variables académicas del 1er semestre
- T2 (Fin 2do Semestre) : T1 + variables académicas del 2do semestre

Preprocesamiento específico para Regrsión Lineal:
- Escalado
- Label Encoding para categóricas
- Target Encoding para 'course'

Pipeline:
1. Carga de datos preprocesados
2. Definición de variables por fase temporal
3. Split estratificado (80/20)
4. Preprocesamiento específico para RL 
5. Entrenamiento con Cross-Validation 5-fold
6. Evaluación en test set
7. Comparación de resultados por fase

## 0. Librerias y configuración

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Preprocesamiento
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelo
from sklearn.linear_model import LogisticRegression

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, precision_recall_curve, average_precision_score
)

# Target Encoding
from category_encoders import TargetEncoder

# MLFLOW
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Seed para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Directorio de salida
OUTPUT_DIR = "../outputs/figures/modelado/baseline_RL/"
OUTPUT_DIR_REPORTES = "../outputs/models/baseline_RL/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR_REPORTES, exist_ok=True)


# =============================================================================
# CONFIGURACIÓN MLFLOW
# =============================================================================

# Nombre del experimento (agrupa todos los runs relacionados)
EXPERIMENT_NAME = "TFM_Dropout_Prediction"

# Tracking URI (por defecto usa carpeta mlruns en el directorio actual)
# Opciones:
#   - "mlruns" (local, carpeta por defecto)
#   - "sqlite:///mlflow.db" (SQLite local)
#   - "http://localhost:5000" (servidor MLflow remoto)
MLFLOW_TRACKING_URI = "../mlruns"

# Configurar MLflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

# Obtener información del experimento
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

print("======================================================================")
print("  CONFIGURACIÓN MLFLOW")
print("======================================================================")
print(f"\nTracking URI: {MLFLOW_TRACKING_URI}")
print(f"Experiment Name: {EXPERIMENT_NAME}")
print(f"Experiment ID: {experiment.experiment_id if experiment else 'Nuevo'}")
print(f"\nPara ver los resultados ejecuta en terminal:")
print(f"   mlflow ui --port 5000")
print(f"   Luego abre: http://localhost:5000")


  CONFIGURACIÓN MLFLOW

Tracking URI: ../mlruns
Experiment Name: TFM_Dropout_Prediction
Experiment ID: 570026133514462797

Para ver los resultados ejecuta en terminal:
   mlflow ui --port 5000
   Luego abre: http://localhost:5000


## 1. Carga de datos preprocesados

In [93]:
# Cargar dataset preprocesado
df_preproc = pd.read_csv('../data/processed/preprocessed_data.csv')
df = df_preproc.copy()  

print(f"Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nTarget binario:")
print(df['target_binario'].value_counts())
print(f"\nRatio: {df['target_binario'].value_counts()[0] / df['target_binario'].value_counts()[1]:.2f}:1")

df.head()

Dataset cargado: 4424 filas x 38 columnas

Target binario:
target_binario
0    3003
1    1421
Name: count, dtype: int64

Ratio: 2.11:1


,application_order,course,daytimeevening_attendance,previous_qualification_grade,admission_grade,displaced,educational_special_needs,debtor,tuition_fees_up_to_date,gender,scholarship_holder,age_at_enrollment,international,curricular_units_1st_sem_credited,curricular_units_1st_sem_enrolled,curricular_units_1st_sem_evaluations,curricular_units_1st_sem_approved,curricular_units_1st_sem_grade,curricular_units_1st_sem_without_evaluations,curricular_units_2nd_sem_credited,curricular_units_2nd_sem_enrolled,curricular_units_2nd_sem_evaluations,curricular_units_2nd_sem_approved,curricular_units_2nd_sem_grade,curricular_units_2nd_sem_without_evaluations,unemployment_rate,inflation_rate,gdp,is_single,application_mode_risk,is_over_23_entry,previous_qualification_risk,mothers_qualification_level,fathers_qualification_level,mothers_occupation_level,fathers_occupation_level,has_unknown_parent_info,target_binario
0,5,171,1,122.0000,127.3000,1,0,0,1,1,0,20,0,0,0,0,0,0.0000,0,0,0,0,0,0.0000,0,10.8000,1.4000,1.7400,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Secundaria,Otro_Trabajo,Otro_Trabajo,0,1
1,1,9254,1,160.0000,142.5000,1,0,0,0,1,0,19,0,0,6,6,6,14.0000,0,0,6,6,6,13.6667,0,13.9000,-0.3000,0.7900,1,Bajo_Riesgo,0,Bajo_Riesgo,Secundaria,Superior,Profesional,Profesional,0,0
2,5,9070,1,122.0000,124.8000,1,0,0,0,1,0,19,0,0,6,0,0,0.0000,0,0,6,0,0,0.0000,0,10.8000,1.4000,1.7400,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Baja,Basica_Baja,Otro_Trabajo,Otro_Trabajo,0,1
3,2,9773,1,122.0000,119.6000,1,0,0,1,0,0,20,0,0,6,8,6,13.4286,0,0,6,10,5,12.4000,0,9.4000,-0.8000,-3.1200,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Basica_Baja,Otro_Trabajo,Profesional,0,0
4,1,8014,0,100.0000,141.5000,0,0,0,1,0,0,45,0,0,6,9,5,12.3333,0,0,6,6,6,13.0000,0,13.9000,-0.3000,0.7900,0,Alto_Riesgo,1,Bajo_Riesgo,Basica_Baja,Basica_Media,Otro_Trabajo,Otro_Trabajo,0,0


In [94]:
# PRUEBA PARA FEATURES
# Cargar dataset preprocesado
df_preproc = pd.read_csv('../data/processed/data_with_financial_academic_crisis_flag.csv')
df = df_preproc.copy()  

print(f"Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nTarget binario:")
print(df['target_binario'].value_counts())
print(f"\nRatio: {df['target_binario'].value_counts()[0] / df['target_binario'].value_counts()[1]:.2f}:1")

df.head()

Dataset cargado: 4424 filas x 39 columnas

Target binario:
target_binario
0    3003
1    1421
Name: count, dtype: int64

Ratio: 2.11:1


,application_order,course,daytimeevening_attendance,previous_qualification_grade,admission_grade,displaced,educational_special_needs,debtor,tuition_fees_up_to_date,gender,scholarship_holder,age_at_enrollment,international,curricular_units_1st_sem_credited,curricular_units_1st_sem_enrolled,curricular_units_1st_sem_evaluations,curricular_units_1st_sem_approved,curricular_units_1st_sem_grade,curricular_units_1st_sem_without_evaluations,curricular_units_2nd_sem_credited,curricular_units_2nd_sem_enrolled,curricular_units_2nd_sem_evaluations,curricular_units_2nd_sem_approved,curricular_units_2nd_sem_grade,curricular_units_2nd_sem_without_evaluations,unemployment_rate,inflation_rate,gdp,is_single,application_mode_risk,is_over_23_entry,previous_qualification_risk,mothers_qualification_level,fathers_qualification_level,mothers_occupation_level,fathers_occupation_level,has_unknown_parent_info,target_binario,financial_academic_crisis_flag
0,5,171,1,122.0000,127.3000,1,0,0,1,1,0,20,0,0,0,0,0,0.0000,0,0,0,0,0,0.0000,0,10.8000,1.4000,1.7400,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Secundaria,Otro_Trabajo,Otro_Trabajo,0,1,0
1,1,9254,1,160.0000,142.5000,1,0,0,0,1,0,19,0,0,6,6,6,14.0000,0,0,6,6,6,13.6667,0,13.9000,-0.3000,0.7900,1,Bajo_Riesgo,0,Bajo_Riesgo,Secundaria,Superior,Profesional,Profesional,0,0,0
2,5,9070,1,122.0000,124.8000,1,0,0,0,1,0,19,0,0,6,0,0,0.0000,0,0,6,0,0,0.0000,0,10.8000,1.4000,1.7400,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Baja,Basica_Baja,Otro_Trabajo,Otro_Trabajo,0,1,1
3,2,9773,1,122.0000,119.6000,1,0,0,1,0,0,20,0,0,6,8,6,13.4286,0,0,6,10,5,12.4000,0,9.4000,-0.8000,-3.1200,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Basica_Baja,Otro_Trabajo,Profesional,0,0,0
4,1,8014,0,100.0000,141.5000,0,0,0,1,0,0,45,0,0,6,9,5,12.3333,0,0,6,6,6,13.0000,0,13.9000,-0.3000,0.7900,0,Alto_Riesgo,1,Bajo_Riesgo,Basica_Baja,Basica_Media,Otro_Trabajo,Otro_Trabajo,0,0,0


## 2. Definición de variables por fase temporal (T0, T1, T2)

In [ ]:
# TARGET
TARGET = 'target_binario'

# -----------------------------------------------------------------------------
# VARIABLES BINARIAS (no requieren encoding, ya son 0/1)
# -----------------------------------------------------------------------------
VARS_BINARIAS_T0 = [
    'daytimeevening_attendance',
    'displaced',
    'educational_special_needs',
    'gender',
    'scholarship_holder',
    'international',
    'is_single'
]

VARS_BINARIAS_T1 = [
    'debtor',
    'tuition_fees_up_to_date',
    #'financial_risk_flag'
    #'early_academic_failure_flag'
    'financial_academic_crisis_flag'
]

# -----------------------------------------------------------------------------
# VARIABLES NUMÉRICAS (requieren escalado)
# -----------------------------------------------------------------------------
VARS_NUMERICAS_T0 = [
    'age_at_enrollment',
    'admission_grade',
    'previous_qualification_grade'
]

VARS_NUMERICAS_T1 = [
    'curricular_units_1st_sem_credited',
    'curricular_units_1st_sem_enrolled',
    'curricular_units_1st_sem_evaluations',
    'curricular_units_1st_sem_approved',
    'curricular_units_1st_sem_grade',
    'curricular_units_1st_sem_without_evaluations',
    'unemployment_rate',
    'inflation_rate',
    'gdp'
]

VARS_NUMERICAS_T2 = [
    'curricular_units_2nd_sem_credited',
    'curricular_units_2nd_sem_enrolled',
    'curricular_units_2nd_sem_evaluations',
    'curricular_units_2nd_sem_approved',
    'curricular_units_2nd_sem_grade',
    'curricular_units_2nd_sem_without_evaluations'
]

# -----------------------------------------------------------------------------
# VARIABLES CATEGÓRICAS AGRUPADAS (requieren One-Hot Encoding)
# -----------------------------------------------------------------------------
VARS_CATEGORICAS_AGRUPADAS_T0 = [
    'application_mode_risk',
    'previous_qualification_risk',
    'mothers_qualification_level',
    'fathers_qualification_level',
    'mothers_occupation_level',
    'fathers_occupation_level'
]

# -----------------------------------------------------------------------------
# VARIABLES CATEGÓRICAS PARA TARGET ENCODING
# -----------------------------------------------------------------------------
VARS_TARGET_ENCODING_T0 = ['course']

# -----------------------------------------------------------------------------
# VARIABLE ORDINAL (se trata como numérica)
# -----------------------------------------------------------------------------
VARS_ORDINALES_T0 = ['application_order']

# =============================================================================
# COMPOSICIÓN DE VARIABLES POR FASE TEMPORAL
# =============================================================================

# T0: Variables disponibles al momento de matrícula
VARS_T0 = (
    VARS_BINARIAS_T0 +
    VARS_NUMERICAS_T0 +
    VARS_CATEGORICAS_AGRUPADAS_T0 +
    VARS_TARGET_ENCODING_T0 +
    VARS_ORDINALES_T0
)

# T1: T0 + variables del 1er semestre
VARS_T1 = (
    VARS_T0 +
    VARS_BINARIAS_T1 +
    VARS_NUMERICAS_T1
)

# T2: T1 + variables del 2do semestre
VARS_T2 = (
    VARS_T1 +
    VARS_NUMERICAS_T2
)

print("================================================================================")
print("  VARIABLES POR FASE TEMPORAL")
print("================================================================================")
print(f"\n T0 (Matrícula): {len(VARS_T0)} variables")
print(f" T1 (Fin 1er Sem): {len(VARS_T1)} variables (+{len(VARS_T1) - len(VARS_T0)})")
print(f" T2 (Fin 2do Sem): {len(VARS_T2)} variables (+{len(VARS_T2) - len(VARS_T1)})")

  VARIABLES POR FASE TEMPORAL

 T0 (Matrícula): 18 variables
 T1 (Fin 1er Sem): 30 variables (+12)
 T2 (Fin 2do Sem): 36 variables (+6)


## 3. Split TRAIN/TEST 

In [ ]:
# Split se hace en totalidad del dataste, posteriormente se seleccionan las variables según la fase temporal para entrenemiento y evaluación

X = df[VARS_T2].copy()
y = df[TARGET].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("================================================================================")
print("  SPLIT TRAIN/TEST")
print("================================================================================")
print(f"\n Entrenamiento: {X_train.shape[0]} ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f" Test:  {X_test.shape[0]} ({X_test.shape[0]/len(df)*100:.1f}%)")

print(f"\nDistribución del target en Entrenamiento:")
print(y_train.value_counts())
print(f"Ratio: {y_train.value_counts()[0] / y_train.value_counts()[1]:.2f}:1")

print(f"\nDistribución del target en Test:")
print(y_test.value_counts())
print(f"Ratio: {y_test.value_counts()[0] / y_test.value_counts()[1]:.2f}:1")

## 4. Funciones de preprocesamiento para RL

In [ ]:
def obtiene_variables_por_fase(fase):
    # Retorna las listas de variables según la fase temporal, retorna diccionario con variables de la fase

    if fase == 'T0':
        return {
            'binarias': VARS_BINARIAS_T0,
            'numericas': VARS_NUMERICAS_T0 + VARS_ORDINALES_T0,
            'categoricas_ohe': VARS_CATEGORICAS_AGRUPADAS_T0,
            'categoricas_te': VARS_TARGET_ENCODING_T0,
            'all': VARS_T0
        }
    elif fase == 'T1':
        return {
            'binarias': VARS_BINARIAS_T0 + VARS_BINARIAS_T1,
            'numericas': VARS_NUMERICAS_T0 + VARS_ORDINALES_T0 + VARS_NUMERICAS_T1,
            'categoricas_ohe': VARS_CATEGORICAS_AGRUPADAS_T0,
            'categoricas_te': VARS_TARGET_ENCODING_T0,
            'all': VARS_T1
        }
    elif fase == 'T2':
        return {
            'binarias': VARS_BINARIAS_T0 + VARS_BINARIAS_T1,
            'numericas': VARS_NUMERICAS_T0 + VARS_ORDINALES_T0 + VARS_NUMERICAS_T1 + VARS_NUMERICAS_T2,
            'categoricas_ohe': VARS_CATEGORICAS_AGRUPADAS_T0,
            'categoricas_te': VARS_TARGET_ENCODING_T0,
            'all': VARS_T2
        }
    else:
        raise ValueError(f"Fase no válida: {fase}. Usar 'T0', 'T1', o 'T2'")


def preprocesamiento_RL_con_transformaciones(X_train, X_test, y_train, fase):
    """    
    Preprocesa los datos para Regresión Logística:
    - StandardScaler para numéricas
    - One-Hot Encoding para categóricas agrupadas
    - Target Encoding para 'course'
    
    Parameters:
    -----------
    X_train, X_test : DataFrames
    y_train : Series
    fase : str ('T0', 'T1', 'T2')
    
    Returns:
    --------
    X_train_processed, X_test_processed, feature_names, preprocessors
    """
    
    variables_fase = obtiene_variables_por_fase(fase)

    variables_fase.values
    
    # Seleccionar solo las variables de la fase
    X_train_fase = X_train[variables_fase['all']].copy()
    X_test_fase = X_test[variables_fase['all']].copy()


    # =========================================================================
    # VARIABLES ZERO-INFLATED
    # =========================================================================
    vars_zero_inflated = [
        'curricular_units_1st_sem_credited',
        'curricular_units_2nd_sem_credited',
        'curricular_units_1st_sem_without_evaluations',
        'curricular_units_2nd_sem_without_evaluations'
    ]
    
    # Filtrar solo las que existen en esta fase
    vars_zero_inflated_fase = [v for v in vars_zero_inflated if v in X_train_fase.columns]
    
    # =========================================================================
    # 1. Aplicar log a variables para variables zero-inflated y variable de edad
    # =========================================================================
    for col in vars_zero_inflated_fase:
        X_train_fase[col] = np.log1p(X_train_fase[col])
        X_test_fase[col] = np.log1p(X_test_fase[col])
        
    # Log transform para age
    X_train_fase['age_at_enrollment'] = np.log1p(X_train_fase['age_at_enrollment'])
    X_test_fase['age_at_enrollment'] = np.log1p(X_test_fase['age_at_enrollment'])
        
        
    # -------------------------------------------------------------------------
    # 2. TARGET ENCODING para 'course' (se hace primero para evitar leakage)
    # -------------------------------------------------------------------------
    te = TargetEncoder(cols=variables_fase['categoricas_te'], smoothing=0.3)
    
    for col in variables_fase['categoricas_te']:
        X_train_fase[col + '_encoded'] = te.fit_transform(X_train_fase[[col]], y_train)[col]
        X_test_fase[col + '_encoded'] = te.transform(X_test_fase[[col]])[col]
        # Eliminar columna original
        X_train_fase = X_train_fase.drop(columns=[col])
        X_test_fase = X_test_fase.drop(columns=[col])
    
    # Actualizar lista de numéricas (agregar las encoded)
    vars_numericas_updated = variables_fase['numericas'] + [col + '_encoded' for col in variables_fase['categoricas_te']]
    
    # -------------------------------------------------------------------------
    # 3. ONE-HOT ENCODING para categóricas agrupadas
    # -------------------------------------------------------------------------
    X_train_fase = pd.get_dummies(
        X_train_fase, 
        columns=variables_fase['categoricas_ohe'], 
        drop_first=True,
        dtype=int
    )
    X_test_fase = pd.get_dummies(
        X_test_fase, 
        columns=variables_fase['categoricas_ohe'], 
        drop_first=True,
        dtype=int
    )
    
    # Alinear columnas (por si hay categorías en test que no están en train)
    X_train_fase, X_test_fase = X_train_fase.align(X_test_fase, join='left', axis=1, fill_value=0)
    
    # -------------------------------------------------------------------------
    # 4. STANDARD SCALER para numéricas
    # -------------------------------------------------------------------------
    scaler = StandardScaler()
    
    # Identificar columnas numéricas en el dataframe procesado
    cols_to_scale = [c for c in vars_numericas_updated if c in X_train_fase.columns]
    
    X_train_fase[cols_to_scale] = scaler.fit_transform(X_train_fase[cols_to_scale])
    X_test_fase[cols_to_scale] = scaler.transform(X_test_fase[cols_to_scale])
    
    # -------------------------------------------------------------------------
    # Guardar información
    # -------------------------------------------------------------------------
    feature_names = X_train_fase.columns.tolist()
    preprocessors = {
        'target_encoder': te,
        'scaler': scaler,
        'feature_names': feature_names
    }
    
    return X_train_fase, X_test_fase, feature_names, preprocessors


## 5. Función de entrenamiento

In [ ]:
def entrena_RL(X_train, y_train, fase, cv_folds=5):

    mlflow.end_run()
    print("================================================================================")
    print(f"  ENTRENAMIENTO REGRESIÓN LOGÍSTICA - FASE {fase}")
    print("================================================================================")
    print(f"\nVariables: {X_train.shape[1]}")
    print(f" Cantidad de registros en dataset de entrenamiento: {X_train.shape[0]}")
    
    # -------------------------------------------------------------------------
    # Cross-Validation con loop manual
    # -------------------------------------------------------------------------
    print(f"\n Cross-Validation ({cv_folds}-fold):")
    
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE)
    
    # Almacenar resultados por fold
    cv_results = {
        'train_accuracy': [], 'test_accuracy': [],
        'train_precision': [], 'test_precision': [],
        'train_recall': [], 'test_recall': [],
        'train_f1': [], 'test_f1': [],
        'train_roc_auc': [], 'test_roc_auc': []
    }

    # Para almacemar resultados de cada fold
    folds_data = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        X_fold_train = X_train.iloc[train_idx]
        X_fold_val = X_train.iloc[val_idx]
        y_fold_train = y_train.iloc[train_idx]
        y_fold_val = y_train.iloc[val_idx]
        
        # Crear y entrenar modelo
        model = LogisticRegression(
            penalty='l2',
            C=1.0,
            solver='lbfgs',
            class_weight='balanced',
            max_iter=1000,
            random_state=RANDOM_STATE
        )
        model.fit(X_fold_train, y_fold_train)
        
        # Predicciones
        y_train_pred = model.predict(X_fold_train)
        y_train_proba = model.predict_proba(X_fold_train)[:, 1]
        y_val_pred = model.predict(X_fold_val)
        y_val_proba = model.predict_proba(X_fold_val)[:, 1]
        
        # Métricas Train
        cv_results['train_accuracy'].append(accuracy_score(y_fold_train, y_train_pred))
        cv_results['train_precision'].append(precision_score(y_fold_train, y_train_pred))
        cv_results['train_recall'].append(recall_score(y_fold_train, y_train_pred))
        cv_results['train_f1'].append(f1_score(y_fold_train, y_train_pred))
        cv_results['train_roc_auc'].append(roc_auc_score(y_fold_train, y_train_proba))
        
        # Métricas Validation
        cv_results['test_accuracy'].append(accuracy_score(y_fold_val, y_val_pred))
        cv_results['test_precision'].append(precision_score(y_fold_val, y_val_pred))
        cv_results['test_recall'].append(recall_score(y_fold_val, y_val_pred))
        cv_results['test_f1'].append(f1_score(y_fold_val, y_val_pred))
        cv_results['test_roc_auc'].append(roc_auc_score(y_fold_val, y_val_proba))
        
   
    # -------------------------------------------------------------------------
    # Resultados por fold
    # -------------------------------------------------------------------------
    print("\n Resultados por fold:")
    for i in range(cv_folds):
        print(f"\n  Fold {i+1}:")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
            train_score = cv_results[f'train_{metric}'][i]
            val_score = cv_results[f'test_{metric}'][i]
            print(f"    {metric:<10} | Train: {train_score:.4f} | Val: {val_score:.4f}")

    mlflow.set_experiment("TFM_Dropout_Prediction")
    with mlflow.start_run(run_name=f"RegresionLogistica_CV5_{fase}"):
        mlflow.set_tag("modelo", 'Baseline - Params por default con feature_financial_academic_crisis_flag')
        mlflow.set_tag("tipo", 'Validacion cruzada')
        mlflow.log_params(model.get_params())

        # Convertir a numpy arrays
        for key in cv_results:
            cv_results[key] = np.array(cv_results[key])


        # -------------------------------------------------------------------------
        # Resumen CV (media ± std)
        # -------------------------------------------------------------------------
        print(f"\n Resumen Cross-Validation:")
        print(f"\n   {'Métrica':<12} {'Train Mean':>12} {'Train Std':>12} {'Val Mean':>12} {'Val Std':>12}")
        print(f"   {'-'*60}")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
            train_mean = cv_results[f'train_{metric}'].mean()
            train_std = cv_results[f'train_{metric}'].std()
            val_mean = cv_results[f'test_{metric}'].mean()
            val_std = cv_results[f'test_{metric}'].std()
            # mlflow
            mlflow.log_metric(f'test_{metric}_mean', val_mean.round(4))
            mlflow.log_metric(f'test_{metric}_std', val_std.round(4))
            
            
            print(f"   {metric:<12} {train_mean:>12.4f} {train_std:>12.4f} {val_mean:>12.4f} {val_std:>12.4f}")

   
    # -------------------------------------------------------------------------
    # Entrenar modelo final en todo el train set
    # -------------------------------------------------------------------------
    model_final = LogisticRegression(
        #penalty='l2',
        penalty='l1',
        C=1.0,
        #solver='lbfgs',
        solver='saga',
        class_weight='balanced',
        max_iter=1000,
        random_state=RANDOM_STATE
    )
    model_final.fit(X_train, y_train)
   
    # -------------------------------------------------------------------------
    # Retornar resultados
    # -------------------------------------------------------------------------
    results = {
        'phase': fase,
        'model': model_final,
        'n_features': X_train.shape[1],
        'cv_results': cv_results,
    }
    
    return results



def resumen_cv(cv_results, fase, modelo):
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

    summary = {
        'modelo': modelo,
        'fase': fase
    }
    
    # -------------------------
    # Métricas de VALIDACIÓN
    # -------------------------
    for metric in metrics:
        summary[f'{metric}_val_mean'] = cv_results[f'test_{metric}'].mean()
        summary[f'{metric}_val_std']  = cv_results[f'test_{metric}'].std()
    
    # -------------------------
    # Métricas de TRAIN
    # -------------------------
    for metric in metrics:
        summary[f'{metric}_train_mean'] = cv_results[f'train_{metric}'].mean()
        summary[f'{metric}_train_std']  = cv_results[f'train_{metric}'].std()

    return pd.DataFrame([summary])

## 6. Modelado FASE T0 (MATRÍCULA)

In [ ]:
# Preprocesamiento para T0
X_train_T0, X_test_T0, features_T0, prep_T0 = preprocesamiento_RL_con_transformaciones(
    X_train, X_test, y_train, fase='T0')


print(f"\n T0 - Dimensiones después del preprocesamiento:")
print(f"   Train: {X_train_T0.shape}")
print(f"   Test:  {X_test_T0.shape}")
print(f"   Features: {len(features_T0)}")

In [ ]:
# Entrenar y evaluar T0
results_T0 = entrena_RL(X_train_T0, y_train, fase='T0')


df_resumen_rl = resumen_cv(
    cv_results=results_T0['cv_results'],
    fase='T0',
    modelo='RegresionLogistica'
)
# Guardar tabla de comparación
df_resumen_rl.to_csv(f"{OUTPUT_DIR_REPORTES}cv_summary_RL.csv", index=False)

print(f"Resultados guardados en: {OUTPUT_DIR_REPORTES}cv_summary_RL.csv")


### Comentarios FASE 0

1. En el resumen de la validación cruzada, se observa que la caída de las métricas en la validacion es en promedio de 0.0094, es pequeña y esperable, en consecuencia, buena capacidad de generalización.
2. Alta estabilidad, la desviación standard en baja en dentrenamiento y validación
3. En la validación, con solo la información al momento de la matrícula se obtiene un rendimiento moderado (recall 0.6948 y AUC 0.7625), lo que indica una capacidad razonable para identificar estudiantes en riesgo en una etapa temprana.
4. Estos resultados validan a la LR con un modelo baseline.


## 7. Modelado FASE T1 (FIN 1ER SEMESTRE)

In [ ]:
# Preprocesamiento para T1
X_train_T1, X_test_T1, features_T1, prep_T1 = preprocesamiento_RL_con_transformaciones(
    X_train, X_test, y_train, fase='T1')

print(f"\n T1 - Dimensiones después del preprocesamiento:")
print(f"   Train: {X_train_T1.shape}")
print(f"   Test:  {X_test_T1.shape}")
print(f"   Features: {len(features_T1)}")

In [ ]:
X_train_T1.hist

In [ ]:
# Entrenar y evaluar T1
results_T1 = entrena_RL(X_train_T1, y_train, fase='T1')

df_resumen_rl_T1 = resumen_cv(
    cv_results=results_T1['cv_results'],
    fase='T1',
    modelo='RegresionLogistica'
)

rl_path = "../outputs/models/baseline_RL/cv_summary_RL.csv"
df_rl = pd.read_csv(rl_path)
df_final = pd.concat([df_rl, df_resumen_rl_T1], ignore_index=True)
csv_path = "cv_summary_RL.csv"

# Guardar tabla de comparación
df_final.to_csv(rl_path, index=False)
print(f"\nResultados guardados en: {OUTPUT_DIR_REPORTES}cv_summary_RL.csv")

### Comentarios FASE 1

1. En el resumen de la validación cruzada, se observa que la caída de las métricas en la validacion es en promedio de 0.0148, la cual si bien aumenta levemente, es pequeña y esperable. Esto sugiere que continua con una buena capacidad de generalización.
2. Alta estabilidad, la desviación standard en baja en entrenamiento y validación, aunque ligeramente superior a T0, puede explicarse con el aumento de variables.
3. En la validación, con la información al momento de la matrícula y el primer semestre, se obtiene una **mejora relevante** con un rendimiento bueno ya que predice a los desertores sobre el 70% y con una mejor capacidad discriminativa (recall 0.7793 y AUC 0.8973) 
4. En esta fase se consolida a la RL como baseline sólido.

## 8. Modelado FASE T2 (FIN 2DO SEMESTRE)

In [ ]:
# Preprocesamiento para T2
X_train_T2, X_test_T2, features_T2, prep_T2 = preprocesamiento_RL_con_transformaciones(
    X_train, X_test, y_train, fase='T2')

print(f"\n T2 - Dimensiones después del preprocesamiento:")
print(f"   Train: {X_train_T2.shape}")
print(f"   Test:  {X_test_T2.shape}")
print(f"   Features: {len(features_T2)}")

In [ ]:
# Entrenar y evaluar T2
results_T2 = entrena_RL(X_train_T2, y_train, fase='T2')


results_T2 = resumen_cv(
    cv_results=results_T2['cv_results'],
    fase='T2',
    modelo='RegresionLogistica'
)

rl_path = "../outputs/models/baseline_RL/cv_summary_RL.csv"
df_rl = pd.read_csv(rl_path)
df_final = pd.concat([df_rl, results_T2], ignore_index=True)
csv_path = "cv_summary_RL.csv"

# Guardar tabla de comparación
df_final.to_csv(rl_path, index=False)
print(f"\nResultados guardados en: {OUTPUT_DIR_REPORTES}cv_summary_RL.csv")

### Comentarios FASE 2

1. En el resumen de la validación cruzada, se observa que la caída de las métricas en la validacion es en promedio de 0.0094, es pequeña y esperable, en consecuencia, buena capacidad de generalización.
2. Alta estabilidad, la desviación standard permanece baja en entrenamiento y validación incluso con la incorporación de las variables de fase T2.
3. En la validación, con la información al momento de la matrícula, primer y segundo semestre, se obtiene una mejora adicional (aunque más moderada) con un recall 0.8136 y AUC 0.9163, lo que indica un alto poder discriminativo.
4. Estos resultados refuerzan el rol de la RL como baseline estable que sirve como punto de comparación con otro más complejos.

## 9. Resumen Final Regresión lineal

In [ ]:
# Mostrar resumen final
df_final = pd.read_csv(f"{OUTPUT_DIR_REPORTES}cv_summary_RL.csv")

print("================================================================================")
print("  RESUMEN Regresion Logistica - CROSS VALIDATION")
print("================================================================================")
print(df_final.to_string(index=False))

## Conclusión

El incremento sostenido del ROC-AUC desde la fase T0 a la fase T2 evidencia de forma empírica que la incorporación de información académica mejora sustancialmente la capacidad discriminativa del modelo. Adiciaonalmente, el recall aumenta logrando una mayor efectividad de detección de estidiantes que desertan. La baja diferencia entre métricas de entrenamiento y validación confirma una adecuada generalización.